# Create a CPU Accelerated YoloV3 Inference Engine
In this section, we will create an inference engine wrapper, a class that will get image data as input, analyze it, and return the analysis result.

## Get Global Variables

First, read the previously stored variables. We need the name of the directory that will be used to store our ML solution files. If this directory does not exist, we will create a directory with a specified directory name.

In [ ]:
import sys
sys.path.append('../../../common')
from env_variables import *

import os
if not os.path.exists(isSolutionPath):
    os.mkdir(isSolutionPath)

## Download the ONNX ML Model
Next, we will download the sample ONNX model to use in the solution. Recall that in this sample, we will be using YoloV3, an object detection ML model. However, you may customize this ML model to fit your needs.

In [ ]:
# Do not change the values below, as the names are embedded into the score.py file.
# If the values are changed, you must also update the score.py content according to new file names
onnxModelFileName = "model.onnx"
onnxLabelFileName = "labels.txt"

onnxModelUrl = "https://media.githubusercontent.com/media/onnx/models/master/vision/object_detection_segmentation/yolov3/model/yolov3-10.onnx"
onnxModelLabels = "https://raw.githubusercontent.com/qqwweee/keras-yolo3/master/model_data/coco_classes.txt"

The code snippet below downloads the YoloV3 model from the [ONNX Model Zoo](https://github.com/onnx/models) repository. 

> <span>[!NOTE]</span>
> The model download URLs frequently change, so if any of the code below fails, please update the source URLs accordingly.

In [ ]:
# Download the model files
import urllib.request
import os

# Download the Yolo V3 pre-trained model
isSolutionModelFilePath = os.path.join(isSolutionPath, onnxModelFileName)
if not os.path.exists(isSolutionModelFilePath):
    res = urllib.request.urlretrieve(onnxModelUrl, isSolutionModelFilePath)
    print("Model file downloaded at: {}".format(isSolutionModelFilePath))
else:
    print("{} already exists here, so not downloading again.".format(isSolutionModelFilePath))
    
# Download the labels of the Yolo V3 pre-trained model
isSolutionModelLabelFilePath = os.path.join(isSolutionPath, onnxLabelFileName)
if not os.path.exists(isSolutionModelLabelFilePath):
    res = urllib.request.urlretrieve(onnxModelLabels, isSolutionModelLabelFilePath)
    print("Labels file downloaded at: {}".format(isSolutionModelLabelFilePath))
else:
    print("{} already exists here, so not downloading again.".format(isSolutionModelLabelFilePath))

## Create Inference Engine Wrapper
Next, we will create a class that with different properties and methods to help scoring and analysing data from an image. This class will also help us specify analytics compute targets, such as CPU, VPU, FPGA, and debugging features.  

> <span style="color:red; font-weight: bold"> [!IMPORTANT] </span>
> Specific to this sample, we are using the YoloV3 model. As you can see from the code below,  
> * The YoloV3 model accepts only raw image bytes with 416 by 416 in size.  
> * Because we expect the `score` method to receive raw bytes of this size (416x416), we have statically coded the image size into our code. If the image is not 416x416 float32, then the code will crash.
> * Why do we statically code the image size? LVA sends video frames to the `score` endpoint. In fact, LVA can send any image size and format. Since LVA can send images with 416x416 size, we do not need to spend additional compute cycles for re-sizing an image.

In [ ]:
%%writefile $isSolutionPath/score.py
import threading
import cv2
import numpy as np
import io
import onnxruntime
import json
import logging
import os
import linecache
import sys

logging.basicConfig(level=logging.DEBUG)

def PrintGetExceptionDetails():
    exType, exValue, exTraceback = sys.exc_info()

    tbFrame = exTraceback.tb_frame
    lineNo = exTraceback.tb_lineno
    fileName = tbFrame.f_code.co_filename

    linecache.checkcache(fileName)
    line = linecache.getline(fileName, lineNo, tbFrame.f_globals)

    exMessage = '[IS] Exception:\n\tFile name: {0}\n\tLine number: {1}\n\tLine: {2}\n\tValue: {3}'.format(fileName, lineNo, line.strip(), exValue)

    logging.info(exMessage)


class MLModel:
    def __init__(self):
        try:
            self._modelFileName = 'model.onnx'
            self._labelFileName = 'labels.txt'
            self._lock = threading.Lock()

            with open(self._labelFileName, "r") as f:
                self._labelList = [l.rstrip() for l in f]
            
            self._onnxSession = onnxruntime.InferenceSession(self._modelFileName)

        except:
            PrintGetExceptionDetails()

    def Preprocess(self, cvImage):
        try:
            imageBlob = cv2.cvtColor(cvImage, cv2.COLOR_BGR2RGB)
            imageBlob = np.array(imageBlob, dtype='float32')
            imageBlob /= 255.
            imageBlob = np.transpose(imageBlob, [2, 0, 1])
            imageBlob = np.expand_dims(imageBlob, 0)

            return imageBlob
        except:
            PrintGetExceptionDetails()

    def Postprocess(self, boxes, scores, indices):
        try:
            detectedObjects = []

            for idx in indices:
                idxTuple = (idx[0], idx[2])
                temp = [i for i in boxes[idxTuple]]  # temp[1, 0, 3, 2] = xmin, ymin, xmax, ymax
                dobj = {
                    "type" : "entity",
                    "entity" : {
                        "tag" : {
                            "value" : self._labelList[idx[1]],
                            "confidence" : str(scores[tuple(idx)])
                        },
                        "box" : {
                            "l" : str(temp[1] / 416),
                            "t" : str(temp[0] / 416),
                            "w" : str((temp[3] - temp[1]) / 416),
                            "h" : str((temp[2] - temp[0]) / 416)
                        }
                    }
                }
                detectedObjects.append(dobj)

            return detectedObjects
            
        except:
            PrintGetExceptionDetails()

    def Score(self, cvImage):
        try:
            with self._lock:
                imageBlob = self.Preprocess(cvImage)
                boxes, scores, indices = self._onnxSession.run(None, {"input_1": imageBlob, "image_shape":np.array([[416, 416]], dtype=np.float32)})
        
            return self.Postprocess(boxes, scores, indices)

        except:
            PrintGetExceptionDetails()

    def About(self):
        return str("<H1>ONNX Version: " + onnxruntime.__version__ + "</H1><BR><H1> App version: v 1.0</H1>")

The `score` method of the inference engine class above will return a dictionary of inferences in the following form:

```
        {
            "entity": {
                "box": {
                    "h": 0.3498992351271351,
                    "l": 0.027884870008988812,
                    "t": 0.6497463818662655,
                    "w": 0.212033897746693
                },
                "tag": {
                    "confidence": 0.9857677221298218,
                    "value": "person"
                }
            },
            "type": "entity"
        },
        {
            "entity": {
                "box": {
                    "h": 0.3593513820482337,
                    "l": 0.6868949751420454,
                    "t": 0.6334065123374417,
                    "w": 0.26539528586647726
                },
                "tag": {
                    "confidence": 0.9851594567298889,
                    "value": "person"
                }
            },
            "type": "entity"
        }
```

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   